In [1]:
import pandas as pd
import sqlite3


In [2]:
# Simulate Orders data
orders_data = {
    'order_id': [101, 102, 103, 104, 105],
    'customer_id': [1, 2, 1, 3, 2],
    'product_id': [10, 11, 12, 10, 11],
    'quantity': [2, 1, 3, 1, 2],
    'order_date': ['2025-08-01', '2025-08-02', '2025-08-03', '2025-08-04', '2025-08-05']
}
df_orders = pd.DataFrame(orders_data)

# Simulate Products data
products_data = {
    'product_id': [10, 11, 12],
    'product_name': ['Laptop', 'Mouse', 'Keyboard'],
    'category': ['Electronics', 'Electronics', 'Peripherals'],
    'price': [1200.00, 25.00, 75.00]
}
df_products = pd.DataFrame(products_data)

# Simulate Customers data
customers_data = {
    'customer_id': [1, 2, 3],
    'customer_name': ['Alice', 'Bob', 'Charlie'],
    'city': ['New York', 'London', 'Paris']
}
df_customers = pd.DataFrame(customers_data)

print("Raw DataFrames created successfully.")

Raw DataFrames created successfully.


In [22]:
conn = sqlite3.connect('ecommerce.db')

In [4]:
cursor = conn.cursor()

# Create Dimension Tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS customer_dim (
        customer_key INTEGER PRIMARY KEY,
        customer_name TEXT,
        city TEXT
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS product_dim (
        product_key INTEGER PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        price REAL
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS date_dim (
        date_key TEXT PRIMARY KEY,
        full_date TEXT,
        year INTEGER,
        month INTEGER
    );
''')

# Create Fact Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales_fact (
        sale_id INTEGER PRIMARY KEY,
        order_id INTEGER,
        customer_key INTEGER,
        product_key INTEGER,
        date_key TEXT,
        quantity INTEGER,
        total_sale_amount REAL,
        FOREIGN KEY(customer_key) REFERENCES customer_dim(customer_key),
        FOREIGN KEY(product_key) REFERENCES product_dim(product_key),
        FOREIGN KEY(date_key) REFERENCES date_dim(date_key)
    );
''')

conn.commit()
print("Database schema created successfully.")

Database schema created successfully.


In [5]:
# Merge all DataFrames to get a single source of truth for the fact table
df_merged = pd.merge(df_orders, df_customers, on='customer_id')
df_merged = pd.merge(df_merged, df_products, on='product_id')

# Add a 'total_sale_amount' column
df_merged['total_sale_amount'] = df_merged['quantity'] * df_merged['price']

# Prepare data for dimension tables
customer_dim_data = df_customers.rename(columns={'customer_id': 'customer_key'})
product_dim_data = df_products.rename(columns={'product_id': 'product_key'})
date_dim_data = df_orders[['order_date']].copy()
date_dim_data['year'] = pd.to_datetime(date_dim_data['order_date']).dt.year
date_dim_data['month'] = pd.to_datetime(date_dim_data['order_date']).dt.month
date_dim_data = date_dim_data.drop_duplicates(subset=['order_date']).rename(columns={'order_date': 'date_key'})
date_dim_data['full_date'] = date_dim_data['date_key']

In [17]:
# Load data into Dimension Tables
customer_dim_data.to_sql('customer_dim', conn, if_exists='replace', index=False)
product_dim_data.to_sql('product_dim', conn, if_exists='replace', index=False)
date_dim_data.to_sql('date_dim', conn, if_exists='replace', index=False)

# Load data into the Fact Table
fact_table_data = df_merged.rename(columns={'customer_id': 'customer_key', 'product_id': 'product_key', 'order_date': 'date_key'})
fact_table_data = fact_table_data[['order_id', 'customer_key', 'product_key', 'date_key', 'quantity', 'total_sale_amount']]
fact_table_data.to_sql('sales_fact', conn, if_exists='replace', index=True, index_label='sale_id')

print("Data loaded into the data warehouse successfully.")

Data loaded into the data warehouse successfully.


In [21]:
# Function to run a query and display results in a DataFrame
def run_query(query):
    return pd.read_sql_query(query, conn)

# Example 1: Total sales per product category
query_category_sales = """
    SELECT
        d.category,
        SUM(f.total_sale_amount) AS total_sales
    FROM sales_fact f
    JOIN product_dim d ON f.product_key = d.product_key
    GROUP BY d.category
    ORDER BY total_sales DESC;
"""
print("Total sales by category:")
print(run_query(query_category_sales))

# Example 2: Top 3 customers by total spend
query_top_customers = """
    SELECT
        c.customer_name,
        SUM(f.total_sale_amount) AS total_spend
    FROM sales_fact f
    JOIN customer_dim c ON f.customer_key = c.customer_key
    GROUP BY c.customer_name
    ORDER BY total_spend DESC
    LIMIT 3;
"""
print("\nTop 3 customers by total spend:")
print(run_query(query_top_customers))

# Close the database connection
conn.close()
print("\nDatabase connection closed.")

Total sales by category:
      category  total_sales
0  Electronics       3675.0
1  Peripherals        225.0

Top 5 customers by total spend:
  customer_name  total_spend
0         Alice       2625.0
1       Charlie       1200.0
2           Bob         75.0

Database connection closed.
